In [1]:
using Rx
using BenchmarkTools

┌ Info: Precompiling Rx [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1273


In [2]:
function regularArray()
    source = [ i for i in 1:1000 ]
    keep = Array{Int, 1}()
    for value in source
        push!(keep, value)
    end
    return keep
end

regularArray (generic function with 1 method)

In [3]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

function observableWithKeepActor()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source, keep)
    return keep.values
end

observableWithKeepActor (generic function with 1 method)

In [4]:
function observableWithLambdaActor()
    source = from([ i for i in 1:1000 ])
    keep = Array{Int, 1}()
    subscribe!(source, LambdaActor{Int}(
        on_next = (d) -> push!(keep, d)
    ))
    return keep
end

observableWithLambdaActor (generic function with 1 method)

In [6]:
@btime regularArray();
@btime observableWithKeepActor();
@btime observableWithLambdaActor();

  6.722 μs (11 allocations: 24.33 KiB)
  7.457 μs (13 allocations: 24.36 KiB)
  398.160 μs (1992 allocations: 55.30 KiB)
